In [1]:
import openmm as mm
import openmm.app as app
import os

In [2]:
name = "methyl_hexanoate"
prmtop = app.AmberPrmtopFile(f"./structure/{name}.prmtop")

In [ ]:
def make_system_and_run_simulation(prmtop, environment: str):
    if environment == "vacuum":
        solvent = None
    elif environment == "OBC2":
        solvent = app.OBC2

    system = prmtop.createSystem(
        nonbondedMethod=app.NoCutoff,
        rigidWater=False,
        implicitSolvent=solvent,
        removeCMMotion=True,
    )

    inpcrd = app.AmberInpcrdFile(f"./structure/{name}.inpcrd")
    with open(f"./structure/{name}_{solvent}.xml", 'w') as file_handle:
        file_handle.write(mm.XmlSerializer.serializeSystem(system))

    ## contruct the context
    integrator = mm.LangevinIntegrator(300, 1, 0.001)
    platform = mm.Platform.getPlatformByName('Reference')
    context = mm.Context(system, integrator, platform)
    context.setPositions(inpcrd.getPositions())
    mm.LocalEnergyMinimizer_minimize(context)

    ## run simulations
    os.makedirs(f"./output/{solvent}/traj", exist_ok = True)
    dcdfile_handle = open(f"./output/{solvent}/traj/traj_md.dcd", 'wb')
    dcdfile = app.DCDFile(dcdfile_handle, prmtop.topology, 1)

    num_steps = int(1e7)
    save_freq = int(1e3)
    num_frames = num_steps//save_freq

    for k in range(num_frames):
        if (k + 1) % 100 == 0:
            print("{} output of total {} frames".format(k, num_frames), flush = True)
            
        integrator.step(save_freq)
        state = context.getState(getPositions = True)
        positions = state.getPositions()
        dcdfile.writeModel(positions)

    dcdfile_handle.close()


'/cluster/tufts/dinglab/xding07/projects_on_github/deepbar/examples/methyl_hexanoate/script'